In [ ]:
from google.colab import drive

# 드라이브를 강제로 다시 마운트합니다
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install yt-dlp -q -U
!pip install pydub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.5 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import subprocess
import os
from pydub import AudioSegment
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import librosa
import soundfile as sf

In [ ]:
title = 'title'
new_title='new_title'
video_id = 'video_id'

# 최종 음성: 분할 후 통합 완료된 음성
# 잘못된 최종 음성 정보 있는 csv 파일
retry_info = [f'/content/drive/MyDrive/NexoChat_share_data/youtube_dataset_30sec/subtitles/info/segement_30sec_info/{new_title}/{video_id}.csv',]

# 잘못된 최종 음성 wav index
retry_nums = [0]

In [ ]:
# 시간 문자열을 밀리초로 변환하는 함수
def time_to_milliseconds_librosa(time_str):
    h, m, s = map(float, time_str.split(':'))
    return int((h * 3600 + m * 60 + s) * 1000)

def process_audio(csv_path):
    video_id = csv_path[-15:-4]
    df = pd.read_csv(csv_path)
    for num, row in enumerate(df.itertuples()):
        new_num = str(num).zfill(5)
        audio_merge_list = json.loads(row.audio_list.replace("'", '"')) # 합쳐야 하는 분할된 오디오 segment 리스트 [00001.wav, 00005.wav ...]
        merged_audio = []

        if int(new_num) not in retry_nums: # 잘못된 최종 음성이 아니라면 무시한다.
            continue

        # print(f'{audio_merge_list} merge 시작')
        # print(int(new_num))

        for audio_file in audio_merge_list:

            tmp = pd.read_csv(f'/content/drive/MyDrive/NexoChat_share_data/youtube_dataset_30sec/subtitles/info/segment_info/{new_title}/{video_id}.csv')

            # 분할해야 하는 오디오 segment 리스트 정보 (00005.wav)
            tmp = tmp[tmp['audio'] == audio_file]

            # 전체 오디오
            full_audio, sr = librosa.load(f'/content/drive/MyDrive/NexoChat_share_data/youtube_dataset/audio/original/{title}/{video_id}.wav', sr=None)

            # print(f'{audio_file} 쪼갬 - {tmp["start_time"].iloc[0]} ~ {tmp["end_time"].iloc[0]}\n')

            # 분할해야 하는 오디오 segment 의 timestamp 정보, timestamp 정보 전처리
            tmp['start_time'] = tmp['start_time'].apply(time_to_milliseconds_librosa)
            tmp['end_time'] = tmp['end_time'].apply(time_to_milliseconds_librosa)

            start_time = int(tmp['start_time'].iloc[0])
            end_time = int(tmp['end_time'].iloc[0])

            start_sample = int(start_time * sr / 1000)
            end_sample = int(end_time * sr / 1000)

            # timestamp 정보에 따라 원본 오디오 분할
            seg = full_audio[start_sample:end_sample]

            # 분할된 오디오를 합친다.
            seg_path = f'/content/{audio_file}'
            sf.write(seg_path, seg, sr)
            merged_audio.extend(seg)

        # 최종 음성 저장
        audio_save_folder = f'/content/drive/MyDrive/NexoChat_share_data/youtube_dataset_30sec/audio/segment/{new_title}/{video_id}'
        os.makedirs(audio_save_folder, exist_ok=True)
        audio_save_path = os.path.join(audio_save_folder, f'{new_num}.wav')
        sf.write(audio_save_path, merged_audio, sr)

        # print(f'{row.sub}\n\n')

# 병렬처리
def main(subtitle_info):
    with ProcessPoolExecutor() as executor:
        list(tqdm(executor.map(process_audio, subtitle_info), total=len(subtitle_info)))

In [ ]:
main(retry_info)

  0%|          | 0/1 [00:00<?, ?it/s]

['00000.wav', '00001.wav', '00002.wav', '00003.wav', '00004.wav', '00005.wav', '00006.wav', '00007.wav', '00008.wav', '00009.wav', '00010.wav', '00011.wav'] merge 시작
0
00000.wav 쪼갬 - 00:01:11.688 ~ 00:01:13.688

00001.wav 쪼갬 - 00:01:15.409 ~ 00:01:19.663

00002.wav 쪼갬 - 00:01:19.663 ~ 00:01:21.832

00003.wav 쪼갬 - 00:01:22.625 ~ 00:01:25.961

00004.wav 쪼갬 - 00:01:25.961 ~ 00:01:29.298

00005.wav 쪼갬 - 00:01:29.715 ~ 00:01:32.259

00006.wav 쪼갬 - 00:01:33.761 ~ 00:01:36.263

00007.wav 쪼갬 - 00:01:36.430 ~ 00:01:38.599

00008.wav 쪼갬 - 00:01:39.350 ~ 00:01:41.018

00009.wav 쪼갬 - 00:01:42.228 ~ 00:01:43.938

00010.wav 쪼갬 - 00:01:44.438 ~ 00:01:45.314

00011.wav 쪼갬 - 00:01:47.891 ~ 00:01:49.652

 일단 성남에 감독을 했으면 하는 생각을 옛날부터 갖고 있었고 제가 몸담았던 팀이기 때문에 항상 애정이 있는 팀이라고 생각을 하고 있었기 때문에 옛날부터 준비를 많이 했었고 어떤 계기가 되어서 성남에 오게 되었는데 성남이 좋은 상황일 때 왔으면 좋은데 어려운 상황에 팀을 맡게 되어서 책임감도 강하게 생각하고 잘 해야겠다는 생각을 많이 가지고 이 팀에 오게 되었습니다 일단 선수들이 처음에 모였을 때는




100%|██████████| 1/1 [00:31<00:00, 31.13s/it]
